In [1]:
import tensorflow as tf
sess = tf.Session()

In [2]:
graph = tf.get_default_graph()

In [3]:
print graph.get_operations()

[]


In [4]:
input_value = tf.constant(1.0)

In [5]:
print graph.get_operations()
print graph.get_operations()[0].node_def
print input_value

name: "Const"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_FLOAT
      tensor_shape {
      }
      float_val: 1.0
    }
  }
}

Tensor("Const:0", shape=(), dtype=float32)


In [6]:
weight = tf.Variable(0.8) #use variable instead of constant!
#variabels support more ops than constants!
for op in graph.get_operations(): print op.name

Const
Variable/initial_value
Variable
Variable/Assign
Variable/read


In [7]:
output = weight * input_value
# multiplication operation
print graph.get_operations()[-1].name
# input values for mul_op.
for inp in graph.get_operations()[-1].inputs: print inp

mul
Tensor("Variable/read:0", shape=(), dtype=float32)
Tensor("Const:0", shape=(), dtype=float32)


In [8]:
# Constants do not need be initialized!
print sess.run(input_value)
try:
    print sess.run(weight)
except:
    print "Variables have to be initialized first!"
init = tf.initialize_all_variables()
sess.run(init)
print sess.run(weight)
print sess.run(output)


1.0
Variables have to be initialized first!
0.8
0.8


In [9]:
# clear graph of all nodes
tf.reset_default_graph()
sess = tf.Session()
for op in tf.get_default_graph().get_operations(): print op
# redefine constants / variables with name field for tensorboard
x = tf.constant(1.0, name="input")
w = tf.Variable(0.8, name="weight")
y = tf.mul(w, x, name="output")
for op in tf.get_default_graph().get_operations(): print op.name
# generate output folder for tensorboard
summary_writer = tf.train.SummaryWriter("log_simple_graph", sess.graph)

input
weight/initial_value
weight
weight/Assign
weight/read
output


In [10]:
# expected output
y_ = tf.constant(0.0, name="expected")
# simple loss function
loss = (y - y_) ** 2
# GDO automatically does backprob and gets the gradients.
optim = tf.train.GradientDescentOptimizer(learning_rate=0.025)
# compute gradients
grads_and_vars = optim.compute_gradients(loss)
sess.run(tf.initialize_all_variables())
print sess.run(grads_and_vars[0])
# do the actual learning
sess.run(optim.apply_gradients(grads_and_vars))
# w should be < 0.8
print sess.run(w)


[1.6, 0.80000001]
0.76


In [11]:
# more compact version of the above:
train_step = tf.train.GradientDescentOptimizer(learning_rate=0.025).minimize(loss)
# diagnostics for tensorboard
summary_y = tf.scalar_summary("output", y)
for i in range(100):
    summary_str = sess.run(summary_y)
    summary_writer.add_summary(summary_str, i)
    sess.run(train_step)
    
print sess.run(w)
print sess.run(y)

0.0044996
0.0044996
